In [ ]:
'''
This notebook was used to extract Java methods from GitHub repositories.
'''

In [ ]:
!pip install pydriller
!pip install javalang

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
#### GHS ####

In [ ]:
!add-apt-repository ppa:git-core/ppa
!apt-get update
!apt-get install git

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/git-core/ppa/ubuntu/ jammy main'
Description:
The most current stable version of Git for Ubuntu.

For release candidates, go to https://launchpad.net/~git-core/+archive/candidate .
More info: https://launchpad.net/~git-core/+archive/ubuntu/ppa
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding key to /etc/apt/trusted.gpg.d/git-core-ubuntu-ppa.gpg with fingerprint F911AB184317630C59970973E363C90F8F1B6217
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get

In [ ]:
import pandas as pd

In [ ]:
from pydriller import Repository
import os
from javalang.parse import parse
from javalang.tree import MethodDeclaration

ModuleNotFoundError: No module named 'pydriller'

In [ ]:
df_res = pd.read_csv('ghs_results.csv')

In [ ]:
repoList = []
for idx,row in df_res.iterrows():
  repoList.append("https://www.github.com/{}".format(row['name']))

#Importance of Collecting High-Quality Data from GitHub Repositories


---


High-quality data is the backbone of training effective and reliable machine learning models, especially for tasks involving code and natural language. GitHub repositories offer a vast amount of real-world, diverse data, making them an invaluable resource for building advanced models like LLMs for software engineering.
However, not all repositories are equally relevant or well-maintained. Selecting only relevant repositories ensures that the da
ta aligns with the task at hand, minimizing noise and improving the model’s performance. This approach also reduces the risk of introducing low-quality or misleading patterns into the training process, resulting in models that are both robust and practical. Thus, we need to collect source code using specific criteria. An helping hand is provided by the following platform:
[GHS](https://seart-ghs.si.usi.ch/).

In [ ]:
from pydriller import Repository
import os
import csv
from javalang.parse import parse
from javalang.tree import MethodDeclaration
import javalang

def extract_methods_from_java(code):
    """
    Extract methods from Java source code using javalang parser.

    Args:
        code (str): The Java source code.

    Returns:
        list: A list of tuples containing method names and their full source code.
    """
    methods = []
    try:
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = javalang.parse.parse(code)
        lines = code.splitlines()

        # Traverse the tree to find method declarations
        for _, node in tree.filter(javalang.tree.MethodDeclaration):
            method_name = node.name

            # Determine the start and end lines of the method
            start_line = node.position.line - 1
            end_line = None

            # Use the body of the method to determine its end position
            if node.body:
                last_statement = node.body[-1]
                if hasattr(last_statement, 'position') and last_statement.position:
                    end_line = last_statement.position.line

            # Extract method code
            if end_line:
                method_code = "\n".join(lines[start_line:end_line+1])
            else:
                # If end_line couldn't be determined, extract up to the end of the file
                method_code = "\n".join(lines[start_line:])

            methods.append((method_name, method_code))

    except Exception as e:
        print(f"Error parsing Java code: {e}")
    return methods


def extract_methods_to_csv_from_master(repo_path, output_csv):
    """
    Extract methods from Java files in the master branch and save them in a CSV file.

    Args:
        repo_path (str): Path to the Git repository.
        output_csv (str): Path to the output CSV file.
    """
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Commit Hash", "File Name", "Method Name", "Method Code", "Commit Link"])

        for commit in Repository(repo_path, only_in_branch="master").traverse_commits():
            print(f"Processing commit: {commit.hash}")

            #We only look into the modified files. In other words, we are looking into the history of the software system by traversing each commit.
            #Various Generative AI methods for SD have been trained on data collected in this way; for example bug fixing.
            for modified_file in commit.modified_files:
                if modified_file.filename.endswith(".java") and modified_file.source_code:
                    methods = extract_methods_from_java(modified_file.source_code)

                    for method_name, method_code in methods:
                        commit_link = f"{repo_path}/commit/{commit.hash}"
                        csv_writer.writerow([commit.hash, modified_file.filename, method_name, method_code, commit_link])

                    print(f"Extracted methods from {modified_file.filename} in commit {commit.hash}")


def extract_methods_to_csv(repo_path, output_csv):
    """
    Extract methods from Java files in a repository and save them in a CSV file.

    Args:
        repo_path (str): Path to the Git repository.
        output_csv (str): Path to the output CSV file.
    """
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Branch Name", "Commit Hash", "File Name", "Method Name", "Method Code", "Commit Link"])

        branch_name = "master"
        for commit in Repository(repo_path, only_in_branch=branch_name).traverse_commits():
            print(f"Processing commit: {commit.hash}")

            for modified_file in commit.modified_files:
                if modified_file.filename.endswith(".java") and modified_file.source_code:
                    methods = extract_methods_from_java(modified_file.source_code)

                    for method_name, method_code in methods:
                        commit_link = f"{repo_path}/commit/{commit.hash}"
                        csv_writer.writerow([branch_name, commit.hash, modified_file.filename, method_name, method_code, commit_link])

                    print(f"Extracted methods from {modified_file.filename} in commit {commit.hash}")

In [ ]:
repoList = repoList[0:4] + [repoList[14]] + [repoList[18]]
print(repoList)

['https://www.github.com/qos-ch/slf4j', 'https://www.github.com/jhy/jsoup', 'https://www.github.com/tootallnate/java-websocket', 'https://www.github.com/mpatric/mp3agic', 'https://www.github.com/oshi/oshi', 'https://www.github.com/dreamhead/moco']


In [ ]:
for repo in repoList:

    fileNameToSave = ''.join(repo.split('github.com')[1:])
    fileNameToSave = fileNameToSave.replace('/','_')

    # Specify the path to the output CSV file
    output_csv_file = "extracted_methods_{}.csv".format(fileNameToSave)
    # Run the extraction
    extract_methods_to_csv_from_master(repo, output_csv_file)


    print(repo)